In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Command to convert .log.labeled files to .csv
# tr '\t' ',' < "(name).log.labeled" | cut -d ',' -f 1-22 > "(name).csv"

# Command to change directory
# cd /mnt/(path name)

# Replace first 7 lines of .csv file (until #types) with the header below
# timestamp,uid,src_ip,src_port,dest_ip,dest_port,protocol,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label

import os
import pandas as pd

folder_path = 'D:\AI Training\ML Model\Test\CSV'
dataframes = []

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path, sep=',')
        dataframes.append(df)

data = pd.concat(dataframes, ignore_index=True)

print(data.head())

           timestamp                 uid         src_ip  src_port  \
0  1540469302.538640  CGm6jB4dXK71ZDWUDh  192.168.1.132   58687.0   
1  1540469197.400159  CnaDAG3n5r8eiG4su2  192.168.1.132    1900.0   
2  1540469385.734089  CUrxU238nt0m6yTgKf  192.168.1.132   32893.0   
3  1540469831.302625  CGQf8t1kjdxB5PHXL4  192.168.1.132   53395.0   
4  1540469831.265405  CUo9DH2QDnCaBIGjkg  192.168.1.132   52801.0   

           dest_ip  dest_port protocol service    duration orig_bytes  ...  \
0     216.239.35.4      123.0      udp       -    0.114184         48  ...   
1  239.255.255.250     1900.0      udp       -  160.367579       7536  ...   
2     216.239.35.8      123.0      udp       -    0.016986         48  ...   
3       2.16.60.82      443.0      tcp       -    0.003497          0  ...   
4      192.168.1.1       53.0      udp     dns    0.036724         34  ...   

  conn_state local_orig local_resp missed_bytes  history orig_pkts  \
0         SF          -          -          0.

In [10]:
print(data.isnull().sum())

timestamp        0
uid              0
src_ip           3
src_port         3
dest_ip          3
dest_port        3
protocol         3
service          3
duration         3
orig_bytes       3
resp_bytes       3
conn_state       3
local_orig       3
local_resp       3
missed_bytes     3
history          3
orig_pkts        3
orig_ip_bytes    3
resp_pkts        3
resp_ip_bytes    3
label            3
dtype: int64
